##### OpenAI Key 환경변수에 저장

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")
# 환경변수에 저장, 현재 프로그램 실행 중일 때까지만
os.environ['OPENAI_API_KEY']=openai_api_key

##### 웹 크로링(네이버 경제신문 내용)

In [ ]:
import bs4
from langchain_classic import hub
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)
docs = loader.load()
docs

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='\n\n무신사, \'2025 트렌드 결산 리포트\' 공개…취향 소비 세분화\n\n무신사는 31일 올 한 해 고객 데이터를 기반으로 소비 흐름을 분석한 \'2025 무신사 트렌드 결산 리포트\'를 공개했다. 리포트에 따르면 러닝코어, 백꾸(가방 꾸미기), 니치 향수 등 개인의 취향과 라이프스타일을 반\n\n\n전자신문\n\n\n\n\n19\n개의 관련뉴스 더보기\n\n\n\n\n\n쿠팡 겨냥한 금감원장 "대형 유통플랫폼, 금융기관 준해서 감독"(종합)\n\n금융소비자가 최우선…주가조작 꿈도 못 꿀 것" 이찬진 금융감독원장은 1일 "금융소비자를 보다 두텁게 보호할 수 있는 제도적 기반과 공적 감독기능을 강화하는 일이 그 무엇보다 선행돼야 한다"고 강조했다. 이 원장은\n\n\n연합뉴스\n\n\n\n\n57\n개의 관련뉴스 더보기\n\n\n\n\n\n2026년 상반기 상반기 건설업 일당 28만원...1.44%↑\n\n2026년 상반기 건설업 전체 직종의 하루 평균 임금이 28만원 수준으로 조사됐다. 대한건설협회가 1일 공개한 상반기 적용 임금실태조사 결과다. 조사 대상은 전국 2000개 공사현장이며 지난해 9월 현장 근로자 임금\n\n\n매일경제\n\n\n\n\n13\n개의 관련뉴스 더보기\n\n\n\n\n\n내연차 팔고 전기차 사면 최대 100만원 지원… 화재 때 최대 100억 보상 보험도 신설\n\n올해부터 출고 후 3년이 지난 내연기관차를 전기차로 바꾸면 최대 100만 원의 지원금이 추가 지급된다. 전기차를 주차 또는 충전하다가 발생한 화재로 제3자에게 피해를 보상해줘야 할 때 최대 100억 원까지 보장하는\n\n\n서울경제\n\n\n\n\n36\n개의 관련뉴스 더보기\n\n\n\n\n\n역대급 불장에…시총 ‘1조 클럽’ 상장사 76개 늘었다\n\n지난해 국내 증시가 역대급 활황을 보인 가운데 ‘1조 클럽’ 상장사

##### 문서 분할

In [ ]:
# Split
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
# splits = text_splitter.split_documents(docs)

# from_tiktoken_encoder 메서드는 텍스트를 분할할 때 tiktoken 인코더를 사용하여 
# 텍스트를 토큰으로 변환한 다음 분할합니다. 
# 이 방법은 특히 OpenAI 모델과 같은 토큰 기반 언어 모델에서 텍스트를 처리할 때 유용합니다. 
# 토큰 단위로 정확한 분할을 가능하게 하므로, 텍스트가 모델의 입력 토큰 제한에 맞도록 더 정밀하게 
# 분할할 수 있습니다.
# 적용 상황: 이 방법은 모델이 토큰 수를 기준으로 텍스트를 처리해야 할 때(예: GPT 모델) 유리합니다.
# 특히 텍스트가 다양한 언어를 포함하거나 복잡한 구문을 가질 때 유용합니다.

# Split: from_tiktoken_encoder(chunk_size=300,chunk_overlap=50)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50
    )

# Make splits
splits = text_splitter.split_documents(docs)
len(splits)

92

##### 임베딩생성 및 Chroma 벡터 스토어 구축 후 저장

In [ ]:
# Embed: from_documents(documents=splits, embedding=OpenAIEmbeddings())
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

##### 문서 검색 

In [37]:
# # 문서 검색 시 더 높은 다양성을 가진 문서를 더 많이 검색합니다.
# # 데이터셋에 유사한 문서가 많을 경우 유용합니다.
# docsearch.as_retriever(
#     search_type="mmr",  # MMR(Maximal Marginal Relevance) 알고리즘을 사용하여 검색
#     search_kwargs={'k': 6, 'lambda_mult': 0.25}  # 상위 6개의 문서를 검색하고 다양성을 높이기 위해 lambda 값을 0.25로 설정
# )

# # MMR 알고리즘이 고려할 문서 수를 더 많이 가져옵니다.
# # 그러나 최종적으로 상위 5개 문서만 반환합니다.
# docsearch.as_retriever(
#     search_type="mmr",  # MMR 알고리즘을 사용하여 검색
#     search_kwargs={'k': 5, 'fetch_k': 50}  # 상위 5개의 문서를 반환하지만, 고려할 문서는 50개로 설정
# )

# # 특정 임계값 이상의 유사도 점수를 가진 문서만 검색합니다.
# docsearch.as_retriever(
#     search_type="similarity_score_threshold",  # 유사도 점수 기반 검색
#     search_kwargs={'score_threshold': 0.8}  # 유사도 점수가 0.8 이상인 문서만 검색
# )

# # 데이터셋에서 가장 유사한 문서 하나만 검색합니다.
# docsearch.as_retriever(search_kwargs={'k': 1})

# MMR 알고리즘이 고려할 문서 수를 더 많이 가져옵니다.
# 그러나 최종적으로 상위 1개 문서만 반환합니다
retriever = vectorstore.as_retriever(
    search_type="mmr", # MMR 알고리즘을 사용하여 검색
    search_kwargs={'k':1,'fetch_k':4} # 상위 1개의 문서를 반환하지만, 고려할 문서는 4개로 설정
)

In [38]:
docs =retriever.invoke("소비자 물가와 관련한 정보를 알려줘")
for doc in docs:
    print(doc.page_content)

올해 소비자물가 지수 상승률 2.1%… 2020년 이후 가장 낮아 빵 5.8%, 햄·베이컨 5.7%, 라면 5% 등 먹거리 물가 크게 올라 서울에 살고 있는 직장인 A(34)씨는 그동안 점심을 주로 햄버거 프랜차이즈


조선비즈




100
개의 관련뉴스 더보기


In [39]:
# Prompt
# prompt = hub.pull("sungwoo/ragbasic")
prompt = hub.pull("rlm/rag-prompt")

In [40]:
# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    formatted = "\n\n".join(doc.page_content for doc in docs)
    return formatted

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("소비자 물가와 관련한 정보를 알려줘")


'올해 소비자물가 지수 상승률은 2.1%로, 2020년 이후 가장 낮은 수치입니다. 특히 빵은 5.8%, 햄·베이컨은 5.7%, 라면은 5% 등 먹거리 물가가 크게 올랐습니다.'

### multiquery + unique-union

In [41]:
#### INDEXING ####

loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)
docs = loader.load()


# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)


In [42]:
# Index
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [43]:
# Multiquery 생성

from langchain_core.prompts import ChatPromptTemplate

template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것입니다. 
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다. 
각 질문은 새 줄로 구분하여 제공하세요. 원본 질문: {question}
"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(model_name="gpt-4o-mini",temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

generated_query = generate_queries.invoke("집값의 향방?")
generated_query

['집값의 미래 전망은 어떻게 될까요?  ',
 '현재 집값의 추세와 앞으로의 변화는 어떤 영향을 받을까요?  ',
 '부동산 시장에서 집값이 오를지 내릴지에 대한 예측은 무엇인가요?  ',
 '경제적 요인들이 집값에 미치는 영향은 어떤 것들이 있을까요?  ',
 '향후 몇 년간 집값의 변동성을 어떻게 분석할 수 있을까요?  ']

In [44]:
from langchain_classic.load import dumps, loads
import json 

def get_unique_union(documents: list[list]):
    """ 고유한 문서들의 합집합을 생성하는 함수입니다. """
    
    # 리스트의 리스트를 평탄화하고, 각 문서를 문자열로 변환(dumps).
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    
    # 중복된 문서를 제거하고 고유한 문서만 남깁니다.
    unique_docs = list(set(flattened_docs))
    
    # 고유한 문서를 원래의 문서 객체로 변환하여 반환합니다.
    return [loads(doc) for doc in unique_docs]

In [45]:
# 사용자 질문 정의
question = "집값의 향방?"

In [ ]:
# 문서 검색 체인을 구성합니다.
# generate_queries: 주어진 질문에 대해 검색 쿼리를 생성합니다.
# retriever.map(): 생성된 쿼리를 바탕으로 관련 문서를 검색합니다.
# get_unique_union: 검색된 문서에서 중복을 제거하고 고유한 문서들을 반환합니다.
retrieval_chain = generate_queries | retriever.map() | get_unique_union

In [ ]:
# 체인을 실행하여 질문에 대한 관련 문서를 검색하고 고유한 문서를 반환합니다.
docs = retrieval_chain.invoke({"question": question})

In [48]:
# 검색된 고유 문서들의 개수를 출력합니다.
print(len(docs))

# 검색된 고유 문서들을 출력합니다.
docs

4


[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='지난해 서울 아파트값이 19년 만에 가장 크게 오른 것으로 집계됐다. 지난 반년 새 대규모 부동산 정책이 잇따라 나왔지만, 외려 ‘똘똘한 한 채’ 현상 가중으로 서울 집값 상승세는 꺾이지 않았다. ‘미친 집값’이란 \n\n\n중앙일보\n\n1시간전'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='지난해 서울 아파트 가격이 8.7% 상승한 것으로 나타났다. 2006년 이후 19년 만에 최고 상승률을 기록한 것이다. 정부의 강력한 부동산 규제가 이어졌지만 서울 송파구와 성동구, 마포구 등 한강벨트 지역으로 수요\n\n\n조선일보\n\n1시간전'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='새해부터는 지방 준공 후 미분양된 아파트를 구입할 시 취득세가 최대 50%까지 감면되고 다주택자 취득세 중과에서 제외된다. 가족 간 주택 등을 유상거래하면서 시가 대비 현저하게 낮은 가격에 양도하면 증여로 간주해 최\n\n\n경향신문\n\n19분전'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='뉴시스\n\n1시간전\n\n\n\n\n\n\n\n\n서울 집값, 마지막 주까지 올랐다…연간 상승률 19년 만 최대')]

In [49]:
# RAG
template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke(question)

"최근 서울 아파트값이 19년 만에 가장 크게 오른 것으로 나타났습니다. 대규모 부동산 정책이 시행되었음에도 불구하고, '똘똘한 한 채' 현상으로 인해 집값 상승세는 지속되고 있습니다. 2025년까지 서울 아파트 매매가격이 47주 연속 상승세를 기록했으며, 특히 송파구와 성동구에서 상승폭이 두드러졌습니다. \n\n새해부터는 지방의 미분양 아파트 구매 시 취득세 감면 혜택이 제공되며, 다주택자에 대한 취득세 중과가 제외되는 등의 정책이 시행됩니다. 이러한 정책들이 집값에 미치는 영향은 앞으로 지켜봐야 할 부분입니다. 전반적으로 집값 상승세가 계속될 것으로 보이며, 특히 수도권 지역에서의 상승폭이 두드러질 것으로 예상됩니다."